# Persistieren von Inhalten

Du weißt jetzt bereits, wie du mit Python (HTML-) Dateien herunterladen und extrahieren kannst. Allerdings solltest du vermeiden, das jedesmal durchzuführen, wenn du Texte analysieren möchtest.

Deswegen musst du die extrahieren Daten irgendwo abspeichern.

## Download von HTML-Dateien

Im ersten Schritt lädst du nochmal die Seite https://www.heise.de/news/Guten-Rutsch-und-ein-gesundes-neues-Jahr-2021-5001311.html herunter, falls sie sich nicht schon in deinem Arbeitsverzeichnis befindet.

In [ ]:
import os
import requests

html_filename = "Guten-Rutsch-und-ein-gesundes-neues-Jahr-2021-5001311.html"
if not os.path.isfile(html_filename):
    print("download")
    r = requests.get(f"https://www.heise.de/news/{html_filename}")
    open(html_filename, 'wb').write(r.content)

Anschließend öffnest du die HTML-Datei, liest sie als Ganzes ein und übergibst den HTML-Code an `BeautifulSoup`. Nun hast du wieder ein `soup`-Objekt zur Verfügung, aus dem du die einzelnen Elemente anschließend extrahieren kannst:

## Extraktion der HTML-Dateien

In [ ]:
from bs4 import BeautifulSoup
html = open(html_filename).read()
soup = BeautifulSoup(html)

Die extrahierten Daten sepicherst du am besten in einem `dict` ab:

In [ ]:
d = {}

Title, Header und Autor haben wir schon im letzten Schritt extrahiert:

In [ ]:
d["title"] = soup.h1.text.strip()
d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]

Ein bisschen komplizierter ist es mit dem Fließtext, weil der in einzelnen `<p>`-Tags steckt. Was du aus anderen Programmiersprachen als `map` kennst, schreibst du in Python als *List Comprehension* (und musst es von rechts nach links lesen). Achtung, `join` funktioniert in Python auch genau andersherum als in praktisch allen anderen Programmiersprachen und erfordert ein Array als *Parameter*. Die Absätze werden so mit *Linfeeds* voneinander getrennt

In [ ]:
d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])

Die Semistruktur aus `<script type="application/ld+json">` kennst du auch schon. Wir übernehmen hier lediglich ein paar Schlüssel, du kannst das nach Wunsch einfach ergänzen:

In [ ]:
import json
ld = json.loads(soup.find("script", type="application/ld+json").string)
for k in ["identifier", "url", "datePublished", "commentCount"]:
    d[k] = ld[0][k]

Jetzt kannst du dir das gesamte `dict` anschauen:

In [ ]:
d

Das sieht gut aus!

## Umwandlung in einen `DataFrame`

Du kannst das `dict` in einen `DataFrame` umwandeln. Bitte beachte, dass dafür eine Liste von `dict` verwendet werden muss. Diese besteht hier nur aus einem einzigen Element. Im gleichen Zug kannst du gleich `identifier` als *Index* für den `DataFrame` auswählen.

In [ ]:
import pandas as pd
articles = pd.DataFrame([d]).set_index("identifier")
articles

Die Darstellung ist nicht ganz optimal. Wenn du Spalte und Zeilen vertauschen willst, kannt du einfach `.T` (für *transpose*) an den `DataFrame` anhängen:

In [ ]:
articles.T

Jetzt wandelst du den `DataFrame` in CSV. Über die eingebauten Methoden geht das ganz einfach:

In [ ]:
articles.to_csv()

Das sieht wegen der Zeilenumbrüche leider nicht so übersichtlich aus. Oftmals ist es bei Textdaten daher eine besser Idee, wenn du JSON als Datenformat verwendest. Das heherrscht `pandas` auch:

In [ ]:
articles.to_json(orient='records')

Für die spätere Vearbeitung ist es oft sehr geschickt, wenn du alles in einer Datenbank abspeicherst, damit du auf deren  eingebaute Selektiosmechanismen zurückgreifen kannst. Dazu brauchst du keinen Datenbank-Server, sondern kannst das einfach mit `sqlite` erledigen:

In [ ]:
import sqlite3
sql = sqlite3.connect("heise-articles.db")
articles.to_sql("articles", sql, index_label="id", if_exists="replace")

Du solltest dich nicht vom Namen `sqlite` täuschen lassen. Damit ist lediglich gemeint, dass die Datenbank filebasier arbeitet und lediglich über eine eingeschränkte Menge an Datentypen verfügt.

Die Selektionsmechnismen sind mit *(Recursive) Common Table Expressions*, *(Correlated) Subqueries* und *Window Functions* absolut auf dem Stand der Technik.

Nebenbei bemerkt ist `sqlite` eine der am besten getesteten Bibliotheken und sicher mehrfach auf jedem Handy installiert.